Способы ввода данных:
1. Ввод с клавиатуры
2. Случайная генерация
3. Импорт файла csv

In [1]:
from prettytable import PrettyTable
from scipy.optimize import linprog
import matplotlib.pyplot as plt
from fractions import Fraction
import random, scipy, csv
import pandas as pd
import numpy as np

In [3]:
try:
    way = int(input('Способ ввода данных: '))

    # Ввод с клавиатуры
    if way == 1:
        sum_strategy_A = int(input('Введите количество стратегий для компании А: '))
        name_rows, name_columns = [], []
        for num in range(sum_strategy_A):
            name = input('Введите название {} стратегии компании А: '.format(num + 1))
            name_rows.append(name)

        sum_strategy_B = int(input('\nВведите количество состояний природы П: '))
        for num in range(sum_strategy_B):
            name = input('Введите название {} состояний природы П: '.format(num + 1))
            name_columns.append(name)

        print('\nВведите матрицу весовых коэффициентов поэлементно:')
        matrix_A = []
        rows = []
        for num_row, row in enumerate(name_rows, start=1):
            for num_column, column in enumerate(name_columns, start=1):
                value = float(input('Введите элемент строки {} столбца {}: '.format(num_row, num_column)))
                rows.append(value)
            matrix_A.append(rows)
            rows = []

        print('\nВведите вектор вероятности поэлементно:')
        matrix_P = []
        for num_column, column in enumerate(name_columns, start=1):
            value = float(input('Введите элемент строки 1 столбца {}: '.format(num_column)))
            matrix_P.append(value)

    elif way == 2:
        name_rows, name_columns = [], []

        # Заполняем матрицу для игрока А
        size_rows = int(input("Введите количество стратегий для компании А: ")) 
        size_cols = size_rows

        # Последовательно генерируем случайные числа типа int в диапазоне (0;1000) и добавляем их в матрицу
        matrix_A = []
        for i in range(size_rows):
            rows = []
            for j in range(size_cols):
                x = random.randint(0, 1000)
                rows.append(x)
            matrix_A.append(rows)

        # список для имен строк и столбцов матрицы:
        name_rows = []
        for i in range(size_rows):
            name_rows.append(i+1)

        # ----------------------------------
        # Заполняем список состояний природы (вероятности)
        sum_strategy_B = int(input('Введите количество состояний природы П: '))
        name = 0
        for num in range(sum_strategy_B):
            name += 1
            name_columns.append(name)

        print('\nВведите вектор вероятности поэлементно:')
        matrix_P = []
        for num_column, column in enumerate(name_columns, start=1):
            value = float(input('Введите элемент строки 1 столбца {}: '.format(num_column)))
            matrix_P.append(value)
    
    elif way == 3:
        matrix_input = []
        matrix_A = []
        file_way = input('Введите путь к файлу: ')
        with open(file_way, 'r', newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=';')
            for row in spamreader:
                matrix_input.append(row)
        
        name_columns = matrix_input[0][1:] # Образование списка с названиями колонок матрицы
        name_rows = []
        for row in range(1, len(matrix_input)):
            if row != len(matrix_input)-1:
                name_rows.append(matrix_input[row][0])
                matrix_A.append(matrix_input[row][1:])
            else:
                matrix_P = matrix_input[row][1:]
        
        matrix_A = [[int(value) for value in row] for row in matrix_A]
        matrix_P = [float(value.replace(',', '.')) for value in matrix_P]
    
    
    matrix_A_DF = matrix_A
    matrix_P_DF = matrix_P

    matrix_A = pd.DataFrame(matrix_A)
    matrix_A.columns = name_columns
    matrix_A.index = name_rows
    print('\nВесовая матрица стратегий игрока A:\n', matrix_A, '\n')
    
    
    matrix_P = [matrix_P]
    matrix_P = pd.DataFrame(matrix_P)
    matrix_P.columns = name_columns
    print('Состояния природы П:\n', matrix_P, '\n\n')
    
    
    
    # ------------------------------------------------------
    # ------------------------------------------------------
    # Оценка по критерию Лапласа

    probability_Laplace = 1/len(name_rows)

    average_Laplace = []
    average = 0
    for row in matrix_A_DF:
        for value in row:
            average += value * probability_Laplace
        average_Laplace.append(average)
        average = 0

    strategy_Laplace = ''
    for num, avrg in enumerate(average_Laplace, start=1):
        if avrg == max(average_Laplace):
            strategy_Laplace += 'A{}-{}; '.format(num, name_rows[num-1])
    print('Оптимальная стратегия игрока А по критерию Лапласа: ', strategy_Laplace[:-2])
    print('Цена игры оптимальной стратегией по критерию Лапласа: ', round(max(average_Laplace), 4))


    # ------------------------------------------------------
    # Оценка по критерию Байеса

    probability_Bayes = matrix_P_DF

    average_Bayes = []
    average = 0
    for row in matrix_A_DF:
        for num_value, value in enumerate(row):
            average += value * probability_Bayes[num_value]
        average_Bayes.append(average)
        average = 0

    strategy_Bayes = ''
    for num, avrg in enumerate(average_Bayes, start=1):
        if avrg == max(average_Bayes):
            strategy_Bayes += 'A{}-{}; '.format(num, name_rows[num-1])

    print('\nОптимальная стратегия игрока А по критерию Байеса: ', strategy_Bayes[:-2])
    print('Цена игры оптимальной стратегией по критерию Байеса: ', round(max(average_Bayes), 4))


    # ------------------------------------------------------
    # Оценка по критерию Гермейера

    probability_Germeier = matrix_P_DF

    matrix_Germeier = []
    rows = []
    num_value = 0
    for num_row, row in enumerate(matrix_A_DF):
        for value in row:
            rows.append(round(value * probability_Germeier[num_value], 3))
            num_value += 1
        matrix_Germeier.append(rows)
        rows = []
        num_value = 0

    # Оптимальная чистая стратегия
    def pure_strategy(matrix_Germeier, name_columns, name_rows, probability_Germeier):
        average_Germeier = []

        for row in matrix_Germeier:
            average = min(row)
            average_Germeier.append(average)


        strategy_Germeier = ''
        for num, avrg in enumerate(average_Germeier, start=1):
            if avrg == max(average_Germeier):
                strategy_Germeier += 'A{}-{}; '.format(num, name_rows[num-1])

        max_strategy = round(max(average_Germeier) / probability_Germeier[matrix_Germeier[average_Germeier.index(max(average_Germeier))].index(max(average_Germeier))], 4) 
        print('\nОптимальная чистая стратегия игрока А по критерию Гермейера: ', strategy_Germeier[:-2])
        print('Цена игры оптимальной чистой стратегией по критерию Гермейера: {}/{}'.format(max(average_Germeier), round(max_strategy, 4)))

    pure_strategy = pure_strategy(matrix_Germeier, name_columns, name_rows, probability_Germeier)


    # Оптимальная смешанная стратегия
    def nash_equilibrium(a, name_columns, name_rows, probability_Germeier, matrix_A_DF):
        c = np.array([1 for x in range(len(a[0]))]) # список с единицами, количество которых равно количеству стратегий игрока A
        b = np.array([1 for x in range(len(a))]) # список с единицами, количество которых равно количеству стратегий игрока B

        # bounds - определяет границы для переменных xi. В нашей программе мы определяем неотрицательные границы (0, None) для каждого xi в связи с условиями задачи.
        bounds = [(0, None)] 
        for x in range(len(a[0]) - 1): 
            bounds += [(0, None)] 

        a_T = -np.transpose(a)
        res = linprog(c, A_ub=a_T, b_ub= -b, bounds=bounds)
        p = [round(res.x[i]/(res.fun), 6) for i in range(len(res.x))] 

        procent = []
        pr = 0
        for num_cols in range(len(name_columns)):
            for num_row, row in enumerate(matrix_A_DF):
                pr += row[num_cols] * p[num_row]
            procent.append(round(pr, 4))
            pr = 0

        sum_procent = []
        for num, value in enumerate(procent):
            sum_procent.append(value * probability_Germeier[num])

        print('Цена игры для игрока А при выборе смешанной оптимальной стратегии по критерию Гермейера: {}/{}'.format(round(1/res.fun, 4), round(sum(sum_procent), 4)))

        # Таблица смешанной стратегии для игрока А
        mix_strategy_A = []
        for i in range(len(p)): 
            mix_strategy_A.append(str(round(p[i]*100)) + '%')

        # Для вывода таблицы PrettyTable
        mytable = PrettyTable()
        mytable.field_names = name_rows # имена полей таблицы
        mytable.add_row(mix_strategy_A) # добавление данных по одной строке за раз
        print('Таблица оптимальных смешанных стратегий для игрока А по критерию Гермейера:\n' + str(mytable))

    a = np.array(matrix_Germeier)
    nash_equilibrium = nash_equilibrium(a, name_columns, name_rows, probability_Germeier, matrix_A_DF)

except:
    print('\nЧто-то пошло не так.. Перезапустите программу')

Способ ввода данных: 1
Введите количество стратегий для компании А: 3
Введите название 1 стратегии компании А: 2
Введите название 2 стратегии компании А: 3
Введите название 3 стратегии компании А: 4

Введите количество состояний природы П: 5
Введите название 1 состояний природы П: 38
Введите название 2 состояний природы П: 24
Введите название 3 состояний природы П: 18
Введите название 4 состояний природы П: 9
Введите название 5 состояний природы П: 2

Введите матрицу весовых коэффициентов поэлементно:
Введите элемент строки 1 столбца 1: 30
Введите элемент строки 1 столбца 2: 30
Введите элемент строки 1 столбца 3: 30
Введите элемент строки 1 столбца 4: 30
Введите элемент строки 1 столбца 5: 5
Введите элемент строки 2 столбца 1: 45
Введите элемент строки 2 столбца 2: 45
Введите элемент строки 2 столбца 3: 20
Введите элемент строки 2 столбца 4: 45
Введите элемент строки 2 столбца 5: -5
Введите элемент строки 3 столбца 1: 35
Введите элемент строки 3 столбца 2: 60
Введите элемент строки 3 с